In [16]:
# pip install fastparquet

# 데이터 경량화 (CSV to Parquet)

- 2019-Oct: 5.67GB -> 2.01GB

In [1]:
import pandas as pd
import numpy as np

# 데이터 로드
path1 = '/Users/hyegwan/Folder/cp2/data/2019-Oct.csv'
path2 = '/Users/hyegwan/Folder/cp2/data/2019-Nov.csv'

data_oct = pd.read_csv(path1)
data_nov = pd.read_csv(path2)

In [2]:
data_oct.to_parquet('/Users/hyegwan/Folder/cp2/data/Oct.parquet')
data_nov.to_parquet('/Users/hyegwan/Folder/cp2/data/Nov.parquet')

In [6]:
df1 = pd.read_parquet('/Users/hyegwan/Folder/cp2/data/Oct.parquet')
df2 = pd.read_parquet('/Users/hyegwan/Folder/cp2/data/Nov.parquet')

In [9]:
df1.head(3)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,None,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,None,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8


In [10]:
df2.head(3)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
2,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,None,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387


# 결측치 처리

In [12]:
df1.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    13515609
brand             6117080
price                   0
user_id                 0
user_session            2
dtype: int64

In [13]:
df2.isnull().sum()

event_time              0
event_type              0
product_id              0
category_id             0
category_code    21898171
brand             9224078
price                   0
user_id                 0
user_session           10
dtype: int64

user_session 결측치 삭제

In [14]:
df1.dropna(subset=['user_session'], inplace=True) 
df2.dropna(subset=['user_session'], inplace=True) 

category_code, brand 결측치를 unknown으로 대체

In [15]:
df1.fillna({'category_code':'unknown', 'brand':'unknown'}, inplace=True)
df2.fillna({'category_code':'unknown', 'brand':'unknown'}, inplace=True)

확인

In [17]:
df1.isnull().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

In [19]:
df2.isnull().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

# event_time 전처리

- UTC 글자 제거
- datetime으로 변경
- GMT+4로 가정하므로 4시간을 더해주기

In [20]:
# UTC 글자 제거
df1['event_time'] = df1['event_time'].apply(lambda x : x[:-4])

In [21]:
# datetime으로 변경
df1['event_time'] = df1['event_time'].astype('datetime64')

In [22]:
# GMT+4 시간대로 변경
import datetime
df1['event_time'] = df1['event_time']+datetime.timedelta(hours=4)

In [23]:
df1.head(3)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 04:00:00,view,44600062,2103807459595387724,unknown,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 04:00:00,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 04:00:01,view,17200506,2053013559792632471,furniture.living_room.sofa,unknown,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8


In [24]:
df1.to_parquet('/Users/hyegwan/Folder/cp2/data/2019-Oct.parquet')